In [ ]:
import matplotlib
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy.spatial.transform import Rotation


def compute_exp_W(sigma, w):
    theta = np.linalg.norm(w)
    if theta < 1e-8:
        return np.eye(3), np.eye(3)
    A = np.exp(sigma) * np.sin(theta)
    B = np.exp(sigma) * np.cos(theta)
    C = (np.exp(sigma) - 1) / sigma
    wx = np.array([
        [0, -w[2], w[1]],
        [w[2], 0, -w[0]],
        [-w[1], w[0], 0]
    ])
    exp_wx = np.eye(3) + wx * (np.sin(theta) / theta) + (wx @ wx) * ((1 - np.cos(theta)) / np.square(theta))
    W = C * np.eye(3) + ((A * sigma + theta * (1 - B)) / (np.square(sigma) + np.square(theta))) * (wx / theta) + (C - ((sigma * (B - 1) + A * theta) / (np.square(sigma) + np.square(theta)))) * ((wx / theta) @ (wx / theta))
    return exp_wx, W

def compute_residuals_jacobian(zi, pi, s, R, t):
    sr = s * R[2]
    srpi = sr @ pi
    ri = zi - (srpi + t[2])
    ds = -srpi
    dw = np.cross(sr, pi)
    dp = -sr
    Ji = np.hstack([ds, dw[:2], dp[2]])
    return ri, Ji

def levenberg_marquardt(z, p, s=1.0, R=np.eye(3), t=np.zeros(3), max_iter=1000):
    
    mu = 0.0
    previous_cost = float('inf')
    
    print('---------------------------------------------')
    print('Starting optimization')
    print('---------------------------------------------')
    
    for iteration in range(max_iter):
        r = []
        J = []
        for i in range(len(z)):
            ri, Ji = compute_residuals_jacobian(z[i], p[i], s, R, t)
            r.append(ri)
            J.append(Ji)
        r = np.stack(r)
        J = np.stack(J)
        
        cost = np.square(r).sum()
        cost_change = np.abs(previous_cost - cost) / cost
        print(f'iter: {iteration}, cost: {np.square(r).sum()}, cost change: {cost_change}')
        
        if cost_change < 1e-14:
            break
        
        delta = -(np.linalg.inv(J.T @ J) @ J.T @ r)
        
        exp_wx, W = compute_exp_W(delta[0], np.hstack([delta[1:3], 0]))
        
        t = s * R @ W @ np.array([0, 0, delta[3]]) + t
        s = s * np.exp(delta[0])
        R = R @ exp_wx
        
        if cost < previous_cost:
            mu *= 0.9
        else:
            mu *= 1.1
        previous_cost = cost
    
    return s, R, t

def scatter(*scatters, **kwargs):
    """Simple function to show a 3D scatter plot.
    """
    colors = ['#0032c6', '#f79646']
    fig = plt.figure(figsize=(8, 8))
    ax = fig.add_subplot(projection='3d')
    for points, label, color in zip(scatters, kwargs['labels'], colors):
        ax.scatter(*points, label=label, alpha=1.0, c=color)
    ax.set_xlim(-1, 1)
    ax.set_ylim(-1, 1)
    ax.set_zlim(-0.5, 1.0)
    ax.set_aspect('equal')
    plt.legend()
    plt.show()

In [ ]:
r = 0.2  # radius of the tube
R = 0.5  # radius of the circle

u = np.arange(0, 2 * np.pi, np.pi / 16)
u, v = np.meshgrid(u, u)
p_gt = np.stack([
    ((R + r * np.cos(v)) * np.cos(u)).flatten(),
    ((R + r * np.cos(v)) * np.sin(u)).flatten(),
    (r * np.sin(v)).flatten()
]).T

dolphin = o3d.io.read_triangle_mesh('dolphin.obj')
p_gt = (Rotation.from_euler('zyx', [0, 90, 90], degrees=True).as_matrix() @ np.asarray(dolphin.vertices).T).T

z_gt = p_gt[:, 2].copy()# + np.random.randn(p_gt.shape[0]) / 20

s_es = 0.4
#R_es, _ = compute_exp_W(1.0, np.random.rand(3)*2)
R_es = np.load('R_es.npy')
#p_es_bias = np.random.rand(3, 1)
p_es_bias = np.load('p_es_bias.npy') / 2
p_es = (s_es * R_es @ p_gt.T + p_es_bias).T

cmap = matplotlib.cm.jet
colors = cmap(np.linspace(0, 1, z_gt.size))

In [ ]:
def scatter(*scatters, **kwargs):
    """Simple function to show a 3D scatter plot.
    """
    plt.rcParams["font.family"] = "Times"
    matplotlib.rc('text', usetex=True)
    matplotlib.rcParams['mathtext.fontset'] = 'custom'
    matplotlib.rcParams['mathtext.rm'] = 'Times'
    matplotlib.rcParams['mathtext.it'] = 'Times:italic'
    matplotlib.rcParams['mathtext.bf'] = 'Times:bold'
    
    colors = ['#000000', '#f79646']
    fig = plt.figure(figsize=(3, 3))
    ax = fig.add_subplot(projection='3d')
    for points, label, color, m in zip(scatters, kwargs['labels'], colors, ['.', '.']):
        ax.scatter(*points, label=label, alpha=1.0, c=color, s=0.05, marker=m)  # Originally, s=1
    ax.set_xlim(-0.5, 0.8)
    ax.set_ylim(-0.2, 1.0)
    ax.set_zlim(-0.2, 0.6)
    ax.set_aspect('equal')
    ax.set_xticks([-0.4, 0.0, 0.4, 0.8])
    ax.set_yticks([-0.1, 0.3, 0.7])
    ax.set_zticks([-0.2, 0.2, 0.6])
    ax.tick_params(axis='z', which='major', pad=2)
    ax.tick_params(axis='x', which='major', pad=-1)
    ax.tick_params(axis='y', which='major', pad=-1)
    ax.view_init(elev=15., azim=-60, roll=0)
    #leg = plt.legend(loc=(0.05, 0.78))
    leg = plt.legend(loc='upper left')
    leg.legendHandles[0]._sizes = [12]
    leg.legendHandles[1]._sizes = [12]
    plt.savefig('depthalignmentbefore.pdf', bbox_inches='tight', pad_inches=0)
    plt.savefig('depthalignmentbefore.png', bbox_inches='tight', pad_inches=0, dpi=300)
    plt.show()

scatter(p_gt.T, p_es.T, labels=['Ground truth', 'Model'])

In [ ]:
s, R, t = levenberg_marquardt(z_gt, p_es)

In [ ]:
1/s

In [ ]:
#%matplotlib qt
fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(111, projection='3d')
new_p = s * R @ p_es.T + t.reshape(3, 1)
real_alt = new_p.copy()
real_alt[2] = z_gt
ax.scatter(*real_alt, c=colors)
ax.scatter(*new_p, c=colors * 0.7)

In [ ]:
#%matplotlib qt
new_p = s * R @ p_es.T + t.reshape(3, 1)

def scatter(*scatters, **kwargs):
    """Simple function to show a 3D scatter plot.
    """
    plt.rcParams["font.family"] = "Times"
    matplotlib.rc('text', usetex=True)
    matplotlib.rcParams['mathtext.fontset'] = 'custom'
    matplotlib.rcParams['mathtext.rm'] = 'Times'
    matplotlib.rcParams['mathtext.it'] = 'Times:italic'
    matplotlib.rcParams['mathtext.bf'] = 'Times:bold'
    
    colors = ['#000000', '#f79646']
    fig = plt.figure(figsize=(3, 3))
    ax = fig.add_subplot(projection='3d')
    for points, label, color, m in zip(scatters, kwargs['labels'], colors, ['.', '.']):
        ax.scatter(*points, label=label, alpha=1, c=color, s=0.05, marker=m)
    ax.set_xlim(-0.5, 0.8)
    ax.set_ylim(-0.2, 1.0)
    ax.set_zlim(-0.2, 0.6)
    ax.set_aspect('equal')
    ax.set_xticks([-0.4, 0.0, 0.4, 0.8])
    ax.set_yticks([-0.1, 0.3, 0.7])
    ax.set_zticks([-0.2, 0.2, 0.6])
    ax.tick_params(axis='z', which='major', pad=2)
    ax.tick_params(axis='x', which='major', pad=-1)
    ax.tick_params(axis='y', which='major', pad=-1)
    ax.view_init(elev=15., azim=-60, roll=0)
    #leg = plt.legend(loc=(0.05, 0.78))
    leg = plt.legend(loc='upper left')
    leg.legendHandles[0]._sizes = [12]
    leg.legendHandles[1]._sizes = [12]
    plt.savefig('depthalignmentafter.pdf', bbox_inches='tight', pad_inches=0)
    plt.savefig('depthalignmentafter.png', bbox_inches='tight', pad_inches=0, dpi=300)
    plt.show()

scatter(p_gt.T, new_p, labels=['Ground truth', 'Aligned model'])